In [ ]:
from app.db import DatabaseConnection
from app.functions import search
from pprint import pprint as pp
from dotenv import load_dotenv 
from app.request import load
from app.constans import *
import pandas as pd
import numpy as np
import ijson
import os


conn = DatabaseConnection(**paramsl)
conn.connect()

# load(request_id="e07aaf8f-a587-42af-a0b3-0abd42a7ffc5")

In [ ]:
input = {
    "origin":{"category": "Footwear"},
    "alternative":{"sku":"100:1000"},
}
data = search(request_id="e07aaf8f-a587-42af-a0b3-0abd42a7ffc5", input=input["origin"], table_name="origin", conn=conn)
print(data)
if data:
    for dt in data:
        print(dt)
else:
    print("no hay data")

In [ ]:
# de esta forma podemos descargar las imagenes mas rapido
import boto3
import concurrent.futures

def download_file(bucket_name, file_name, output_directory):
    s3 = boto3.client('s3')
    output_path = output_directory + '/' + file_name
    s3.download_file(bucket_name, file_name, output_path)
    print(f"Archivo descargado: {file_name}")

def download_files_from_s3(bucket_name, directory, output_directory):
    s3 = boto3.client('s3')
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=directory)
    objects = response['Contents']
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for obj in objects:
            file_name = obj['Key']
            if file_name.endswith('/'):
                continue  # Ignorar directorios
            futures.append(executor.submit(download_file, bucket_name, file_name, output_directory))
        
        # Esperar a que se completen todas las descargas
        concurrent.futures.wait(futures)

# Ejemplo de uso
bucket_name = 'hydrahi4ai'
directory = 'directorio_dentro_del_bucket'
output_directory = '/ruta/directorio_salida'

download_files_from_s3(bucket_name, directory, output_directory)